In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/short-jokes/shortjokes.csv


In [5]:
!git clone https://github.com/kamalkraj/minGPT-TF.git

Cloning into 'minGPT-TF'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 64 (delta 30), reused 45 (delta 17), pack-reused 0
Unpacking objects: 100% (64/64), done.


In [ ]:
!cd ./minGPT-TF

In [ ]:
!pwd 
# ./

In [ ]:
import os
os.chdir('./minGPT-TF')

In [6]:
! pip install fastprogress==0.2.3

  Attempting uninstall: fastprogress
    Found existing installation: fastprogress 1.0.0
    Uninstalling fastprogress-1.0.0:
      Successfully uninstalled fastprogress-1.0.0
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

fastai 2.0.16 requires fastprogress>=0.2.4, but you'll have fastprogress 0.2.3 which is incompatible.


In [ ]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [8]:
import math
import numpy as np
import tensorflow as tf
from mingpt.model import GPT, GPTConfig

In [9]:
class CharDataset:

    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return math.ceil(len(self.data) / (self.block_size + 1))

    def __iter__(self):
        # we're actually going to "cheat" and pick a spot in the dataset at 
        for _ in range(self.__len__()):
            i = np.random.randint(0, len(self.data) - (self.block_size + 1))
            chunk = self.data[i:i+self.block_size+1]
            dix = [self.stoi[s] for s in chunk]
            x = tf.convert_to_tensor(dix[:-1], dtype=tf.int32)
            y = tf.convert_to_tensor(dix[1:], dtype=tf.int32)
            yield x, y
    
    __call__ = __iter__

In [10]:
block_size = 128 

In [ ]:
# text = open('input.txt', 'r').read()
# train_dataset_gen = CharDataset(text, block_size) 

In [ ]:
# text

In [ ]:
# train_dataset = tf.data.Dataset.from_generator(train_dataset_gen,(tf.int32,tf.int32))

In [ ]:
# from mingpt.model import GPT, GPTConfig
# mconf = GPTConfig(train_dataset_gen.vocab_size, train_dataset_gen.block_size,
#                   n_layer=8, n_head=8, n_embd=512)

In [ ]:
# from mingpt.trainer import Trainer, TrainerConfig

# # initialize a trainer instance and kick off training
# tconf = TrainerConfig(max_epochs=10, batch_size=128, learning_rate=6e-4,
#                       lr_decay=True, warmup_tokens=512*20, final_tokens=200*len(train_dataset_gen)*block_size,
#                       num_workers=4)
# trainer = Trainer(GPT, mconf, train_dataset, len(train_dataset_gen), None, None, tconf)

In [ ]:
# trainer.train()

In [ ]:
# # alright, let's sample some character-level
# from mingpt.utils import sample

# context = "O God, O God!"
# x = tf.convert_to_tensor([train_dataset_gen.stoi[s] for s in context], dtype=tf.int32)[None,...]
# y = sample(trainer.model, x, 2000, temperature=0.9, sample=True, top_k=5)[0]
# completion = ''.join([train_dataset_gen.itos[int(i)] for i in y])
# print(completion)

In [ ]:
# os.chdir('/kaggle/working')

In [2]:
import pandas as pd
txt = pd.read_csv('../input/short-jokes/shortjokes.csv')
txt.head()

,ID,Joke
0,1,"[me narrating a documentary about narrators] ""..."
1,2,Telling my daughter garlic is good for you. Go...
2,3,I've been going through a really rough period ...
3,4,"If I could have dinner with anyone, dead or al..."
4,5,Two guys walk into a bar. The third guy ducks.


In [3]:
txt1= []

for i in txt['Joke']:
    txt1.append(i)

In [4]:
txt2 = str(txt1)


In [5]:
type(txt2)

str

In [7]:
import os
os.chdir('./minGPT-TF')

In [11]:
train_dataset_gen = CharDataset(txt2, block_size) 

data has 22508167 characters, 95 unique.


In [12]:
train_dataset = tf.data.Dataset.from_generator(train_dataset_gen,(tf.int32,tf.int32))

In [13]:
from mingpt.model import GPT, GPTConfig
mconf = GPTConfig(train_dataset_gen.vocab_size, train_dataset_gen.block_size,
                  n_layer=8, n_head=8, n_embd=512)

In [14]:
from mingpt.trainer import Trainer, TrainerConfig


tconf = TrainerConfig(max_epochs=10, batch_size=128, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=200*len(train_dataset_gen)*block_size,
                      num_workers=4)
trainer = Trainer(GPT, mconf, train_dataset, len(train_dataset_gen), None, None, tconf)

In [15]:
trainer.train()

epoch 1: train loss 231.14236. lr 5.999630e-04
epoch 2: train loss 169.85301. lr 5.998520e-04
epoch 3: train loss 158.83913. lr 5.996671e-04
epoch 4: train loss 152.69626. lr 5.994082e-04
epoch 5: train loss 148.66898. lr 5.990753e-04
epoch 6: train loss 145.65729. lr 5.986688e-04
epoch 7: train loss 143.14734. lr 5.981885e-04
epoch 8: train loss 140.98088. lr 5.976346e-04
epoch 9: train loss 139.33232. lr 5.970074e-04
epoch 10: train loss 137.86714. lr 5.963068e-04


In [21]:

from mingpt.utils import sample

context = "love"
x = tf.convert_to_tensor([train_dataset_gen.stoi[s] for s in context], dtype=tf.int32)[None,...]
y = sample(trainer.model, x, 500, temperature=0.9, sample=True, top_k=5)[0]
completion = ''.join([train_dataset_gen.itos[int(i)] for i in y])
print(completion)

love the whole world of parenting is that I have anything to dry.", "I can never find out if anyone will show up that into the field that isn't asking for some time... That isn't my favorite currency in my family.", "How do you make hormone a black man fly in a vending man? You don't have to see if he wants to check his mind out!", 'What do you call a gay girl? Sir Cumference.', 'What do you call a guy in school? An elevator.', "If you love something you can stand on a woman it's still there. If you


In [23]:

from mingpt.utils import sample

context = "Tanul is crazy"
x = tf.convert_to_tensor([train_dataset_gen.stoi[s] for s in context], dtype=tf.int32)[None,...]
y = sample(trainer.model, x, 100, temperature=0.9, sample=True, top_k=5)[0]
completion = ''.join([train_dataset_gen.itos[int(i)] for i in y])
print(completion)

Tanul is crazy if you can\'t be a constant president in the bed is like a big penis... I don\'t want to say anythi


In [25]:

from mingpt.utils import sample

context = "There was no one"
x = tf.convert_to_tensor([train_dataset_gen.stoi[s] for s in context], dtype=tf.int32)[None,...]
y = sample(trainer.model, x, 90, temperature=0.9, sample=True, top_k=5)[0]
completion = ''.join([train_dataset_gen.itos[int(i)] for i in y])
print(completion)

There was no one was gonna take a couple.', 'Two men walk into a bar. The one says, "One is a straight man


In [26]:

from mingpt.utils import sample

context = "How do you feel when you lie to me"
x = tf.convert_to_tensor([train_dataset_gen.stoi[s] for s in context], dtype=tf.int32)[None,...]
y = sample(trainer.model, x, 64, temperature=0.9, sample=True, top_k=5)[0]
completion = ''.join([train_dataset_gen.itos[int(i)] for i in y])
print(completion)

How do you feel when you lie to me? Your mom.', 'What do you call a gay man? A baby in a stranger'


In [27]:

from mingpt.utils import sample

context = "You are a Dick "
x = tf.convert_to_tensor([train_dataset_gen.stoi[s] for s in context], dtype=tf.int32)[None,...]
y = sample(trainer.model, x, 64, temperature=0.9, sample=True, top_k=5)[0]
completion = ''.join([train_dataset_gen.itos[int(i)] for i in y])
print(completion)

You are a Dickshell, but I don't want to be a giraffe. Worried that I don't th


In [28]:

from mingpt.utils import sample

context = "That cat "
x = tf.convert_to_tensor([train_dataset_gen.stoi[s] for s in context], dtype=tf.int32)[None,...]
y = sample(trainer.model, x, 64, temperature=0.9, sample=True, top_k=5)[0]
completion = ''.join([train_dataset_gen.itos[int(i)] for i in y])
print(completion)

That cat show."', 'How do I take the toilet? Wait, in they stand up and p


In [29]:

from mingpt.utils import sample

context = "This is fucking insane"
x = tf.convert_to_tensor([train_dataset_gen.stoi[s] for s in context], dtype=tf.int32)[None,...]
y = sample(trainer.model, x, 64, temperature=0.9, sample=True, top_k=5)[0]
completion = ''.join([train_dataset_gen.itos[int(i)] for i in y])
print(completion)

This is fucking insane.", 'What do you call a cheap girl with no fingers? Massive.', '


In [32]:

from mingpt.utils import sample

context = "I can crack jokes do you want "
x = tf.convert_to_tensor([train_dataset_gen.stoi[s] for s in context], dtype=tf.int32)[None,...]
y = sample(trainer.model, x, 100, temperature=0.9, sample=True, top_k=5)[0]
completion = ''.join([train_dataset_gen.itos[int(i)] for i in y])
print(completion)

I can crack jokes do you want to become an Asian girl who's had a hard time making a sense of humor.", "Why do the birds always wa
